In [ ]:
!pip install ultralytics pycocotools requests tqdm pyyaml

In [ ]:
from pycocotools.coco import COCO
import requests
import os
from tqdm import tqdm
import json

# Diretórios
os.makedirs('custom_coco/images', exist_ok=True)
os.makedirs('custom_coco/annotations', exist_ok=True)

# Baixar annotations val2017 (se ainda não baixado)
if not os.path.exists('custom_coco/annotations/instances_val2017.json'):
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip -q annotations_trainval2017.zip
    !mv annotations/instances_val2017.json custom_coco/annotations/instances_val2017.json

# Script de filtro corrigido
class CocoFilter:
    def __init__(self, json_path, imgs_dir, categories=['person', 'car']):
        self.coco = COCO(json_path)
        self.json_path = json_path
        self.imgs_dir = imgs_dir
        self.categories = categories
        # Obter IDs de categorias
        self.cat_ids = self.coco.getCatIds(catNms=self.categories)
        print(f"IDs das categorias {self.categories}: {self.cat_ids}")
        # Obter imagens com PELO MENOS UMA categoria (OR)
        img_ids = set()
        for cat_id in self.cat_ids:
            img_ids.update(self.coco.getImgIds(catIds=[cat_id]))
        self.img_ids = list(img_ids)
        self.images = self.coco.loadImgs(self.img_ids)
        print(f"Total de imagens antes do filtro: {len(self.coco.getImgIds())}")
        print(f"Total de imagens após filtro: {len(self.img_ids)}")

    def download_images(self):
        for im in tqdm(self.images, desc="Baixando imagens"):
            img_path = os.path.join(self.imgs_dir, im['file_name'])
            if not os.path.exists(img_path):
                try:
                    img_data = requests.get(im['coco_url'], timeout=10).content
                    with open(img_path, 'wb') as handler:
                        handler.write(img_data)
                except Exception as e:
                    print(f"Erro ao baixar {im['file_name']}: {e}")

    def filter_json(self, output_json_path):
        new_json = {
            'info': self.coco.dataset['info'],
            'licenses': self.coco.dataset['licenses'],
            'categories': [
                {'id': 0, 'name': 'person', 'supercategory': 'person'},
                {'id': 1, 'name': 'car', 'supercategory': 'vehicle'}
            ],
            'images': self.images,
            'annotations': []
        }

        # Mapear IDs originais do COCO para novos IDs (0: person, 1: car)
        cat_map = {self.coco.getCatIds(['person'])[0]: 0, self.coco.getCatIds(['car'])[0]: 1}
        print(f"Mapeamento de categorias: {cat_map}")

        # Filtrar anotações
        ann_ids = self.coco.getAnnIds(imgIds=self.img_ids, catIds=self.cat_ids)
        anns = self.coco.loadAnns(ann_ids)
        for ann in anns:
            if ann['category_id'] in cat_map:
                ann['category_id'] = cat_map[ann['category_id']]
                new_json['annotations'].append(ann)

        # Remapear IDs de imagens
        img_map = {img['id']: i+1 for i, img in enumerate(new_json['images'])}
        for img in new_json['images']:
            img['id'] = img_map[img['id']]
        for ann in new_json['annotations']:
            ann['image_id'] = img_map[ann['image_id']]
            ann['id'] = new_json['annotations'].index(ann) + 1

        with open(output_json_path, 'w') as f:
            json.dump(new_json, f)

# Executar
filter = CocoFilter('custom_coco/annotations/instances_val2017.json', 'custom_coco/images')
filter.download_images()
filter.filter_json('custom_coco/annotations/filtered_val2017.json')
print("Filtro concluído.")

# Verificar JSON
with open('custom_coco/annotations/filtered_val2017.json', 'r') as f:
    data = json.load(f)
    print(f"Imagens: {len(data['images'])}, Anotações: {len(data['annotations'])}")
    print(f"Categorias: {data['categories']}")

In [ ]:
import json
import os
from pathlib import Path
import shutil

def convert_coco_to_yolo(json_path, output_dir, img_dir):
    # Remove destination directory if it exists to ensure a clean state
    if os.path.exists(output_dir):
        print(f"Removing existing output directory: {output_dir}")
        shutil.rmtree(output_dir)
    print(f"Creating output directory: {output_dir}")
    os.makedirs(output_dir, exist_ok=True)

    with open(json_path, 'r') as f:
        coco = json.load(f)

    # Mapear IDs de categorias (0: person, 1: car)
    cat_map = {cat['id']: idx for idx, cat in enumerate(coco['categories'])}
    print(f"Mapeamento de categorias: {cat_map}")

    # Mapear imagens por ID
    img_map = {img['id']: img for img in coco['images']}

    # Agrupar anotações por image
    anns_by_img = {}
    for ann in coco['annotations']:
        img_id = ann['image_id']
        if img_id not in anns_by_img:
            anns_by_img[img_id] = []
        anns_by_img[img_id].append(ann)

    # Convert to YOLO format
    converted_count = 0
    for img_id, anns in anns_by_img.items():
        img_info = img_map.get(img_id)
        if not img_info:
            continue
        img_w, img_h = img_info['width'], img_info['height']
        img_name = img_info['file_name']

        # Label file
        label_path = os.path.join(output_dir, Path(img_name).stem + '.txt')
        with open(label_path, 'w') as f:
            for ann in anns:
                cat_id = cat_map.get(ann['category_id'])
                if cat_id is None:
                    continue
                bbox = ann['bbox']  # [x_min, y_min, width, height]
                x_center = (bbox[0] + bbox[2] / 2) / img_w
                y_center = (bbox[1] + bbox[3] / 2) / img_h
                w = bbox[2] / img_w
                h = bbox[3] / img_h
                f.write(f"{cat_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
        converted_count += 1

    print(f"Conversion completed. {converted_count} label files created in {output_dir}")

# Execute
json_path = 'custom_coco/annotations/filtered_val2017.json'
output_dir = 'custom_coco/images/labels' # Modified to output labels directly to the correct location
img_dir = 'custom_coco/images'
convert_coco_to_yolo(json_path, output_dir, img_dir)

# Verify a label
label_dir = 'custom_coco/images/labels' # Modified to check the new output directory
labels = os.listdir(label_dir)
if labels:
    print(f"Contents of {label_dir} after conversion:")
    print(labels[:10]) # Print only the first 10 files if there are many
    print(f"Total files in {label_dir} after conversion: {len(labels)}")
    with open(os.path.join(label_dir, labels[0]), 'r') as f:
        print(f"Content of {labels[0]}:\n{f.read()}")
else:
    print("No labels created! Check the JSON.")

In [ ]:
import json
import os
from tqdm import tqdm


annotations_file = '/content/custom_coco/annotations/instances_val2017.json'


labels_output_dir = '/content/custom_coco/labels'

coco_classes_to_extract = {
    1: 'person',
    3: 'car'
}


yolo_class_mapping = {
    1: 0,
    3: 1
}


# Garante que o diretório de saída exista
os.makedirs(labels_output_dir, exist_ok=True)

# Carrega o arquivo de anotações do COCO
with open(annotations_file, 'r') as f:
    data = json.load(f)

# Cria um dicionário para acesso rápido às informações da imagem (largura, altura)
images_info = {img['id']: {'width': img['width'], 'height': img['height'], 'file_name': img['file_name']} for img in data['images']}

# Dicionário para armazenar as anotações de cada imagem
image_annotations = {}

print("Processando anotações do COCO...")
# Itera sobre todas as anotações do arquivo JSON
for ann in tqdm(data['annotations']):
    category_id = ann['category_id']

    # Se a classe da anotação for uma das que queremos extrair...
    if category_id in coco_classes_to_extract:
        image_id = ann['image_id']

        if image_id not in image_annotations:
            image_annotations[image_id] = []

        # Converte as coordenadas do COCO para o formato YOLO
        coco_bbox = ann['bbox'] # [x_min, y_min, width, height]
        img_info = images_info[image_id]
        img_width, img_height = img_info['width'], img_info['height']

        # Calcula o centro do bounding box
        x_center = coco_bbox[0] + coco_bbox[2] / 2
        y_center = coco_bbox[1] + coco_bbox[3] / 2

        # Normaliza as coordenadas (divide pela largura/altura da imagem)
        x_center_norm = x_center / img_width
        y_center_norm = y_center / img_height
        width_norm = coco_bbox[2] / img_width
        height_norm = coco_bbox[3] / img_height

        # Obtém o novo ID da classe para o YOLO
        yolo_class_id = yolo_class_mapping[category_id]

        # Adiciona a anotação formatada à lista da imagem
        image_annotations[image_id].append(f"{yolo_class_id} {x_center_norm} {y_center_norm} {width_norm} {height_norm}")

print("Gerando arquivos .txt para o YOLO...")

for image_id, annotations in tqdm(image_annotations.items()):
    img_info = images_info[image_id]
    file_name_without_ext = os.path.splitext(img_info['file_name'])[0]
    output_path = os.path.join(labels_output_dir, f"{file_name_without_ext}.txt")

    with open(output_path, 'w') as f:
        f.write("\n".join(annotations))

print(f"Conversão concluída! {len(image_annotations)} arquivos de label gerados em '{labels_output_dir}'.")

In [ ]:
import yaml

base_path = '/content/custom_coco'


data = {
    'path': base_path,
    'train': 'images',
    'val': 'images',

    'nc': 2,
    'names': ['person', 'car']
}


yaml_file_path = 'custom_coco.yaml'

with open(yaml_file_path, 'w') as f:

    yaml.dump(data, f, default_flow_style=False, sort_keys=False)

print(f"Arquivo '{yaml_file_path}' criado com sucesso!")
print("\nConteúdo do arquivo:")
with open(yaml_file_path, 'r') as f:
    print(f.read())

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model.train(data='custom_coco.yaml', epochs=5, imgsz=640, batch=16)

In [ ]:
from ultralytics import YOLO
from PIL import Image
import IPython.display

IPython.display.clear_output()

model_path = '/content/JSON2YOLO/runs/detect/train13/weights/best.pt'

image_to_test = 'https://ultralytics.com/images/bus.jpg'


try:
    model = YOLO(model_path)
    print(f"Modelo '{model_path}' carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}")
    print("Verifique se o caminho para o arquivo 'best.pt' está correto.")

try:
    results = model.predict(source=image_to_test)
    print(f"Predição realizada na imagem: '{image_to_test}'")


    for r in results:
        im_array = r.plot()
        im = Image.fromarray(im_array[..., ::-1])
        display(im)

except Exception as e:
    print(f"Erro durante a predição: {e}")
    print("Verifique se o caminho para a imagem de teste está correto.")